In [ ]:
#!pip install git+https://github.com/LLaVA-VL/LLaVA-NeXT.git

In [ ]:
#!pip install einops accelerate open-clip-torch

In [ ]:
#!pip install flash-attn --no-build-isolation

In [ ]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path, process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, IGNORE_INDEX
from llava.conversation import conv_templates, SeparatorStyle

In [ ]:
from PIL import Image
import requests
import copy
import torch
import sys
import warnings

In [ ]:
warnings.filterwarnings("ignore")
pretrained = "lmms-lab/llava-onevision-qwen2-72b-ov"
model_name = "llava_qwen"
device = "cuda"
device_map = "auto"

In [ ]:
tokenizer, model, image_processor, max_length = load_pretrained_model(pretrained, None, model_name, device_map=device_map)

In [ ]:
model.eval()

In [ ]:
urls = [
 'https://samplebook.photos/img/4786a.jpg',
 'https://samplebook.photos/img/284v.jpg',
 'https://samplebook.photos/img/3193c.jpg',
 'https://samplebook.photos/img/948d.jpg',
 'https://samplebook.photos/img/3191i.jpg',
 'https://samplebook.photos/img/4788oo.jpg',
 'https://samplebook.photos/img/2086d.jpg',
 'https://samplebook.photos/img/4781r.jpg',
 'https://samplebook.photos/img/5521n.jpg',
 'https://samplebook.photos/img/2073cc.jpg',
 'https://samplebook.photos/img/2085b.jpg',
 'https://samplebook.photos/img/4782c.jpg',
 'https://samplebook.photos/img/4783v.jpg',
 'https://samplebook.photos/img/4912gg.jpg',
 'https://samplebook.photos/img/304h.jpg',
 'https://samplebook.photos/img/2078i.jpg',
 'https://samplebook.photos/img/2077q.jpg',
 'https://samplebook.photos/img/3192h.jpg',
 'https://samplebook.photos/img/904x.jpg',
 'https://samplebook.photos/img/4792b.jpg',
 'https://samplebook.photos/img/5301p.jpg',
 'https://samplebook.photos/img/5207j.jpg',
 'https://samplebook.photos/img/1007p.jpg',
 'https://samplebook.photos/img/17l.jpg',
 'https://samplebook.photos/img/286t.jpg',
 'https://samplebook.photos/img/287g.jpg',
 'https://samplebook.photos/img/3189j.jpg',
 'https://samplebook.photos/img/4911d.jpg',
 'https://samplebook.photos/img/4785a.jpg',
 'https://samplebook.photos/img/4793a.jpg'
]

In [ ]:
prompt = 'There are two objects in this image. One is a color checker, and the other is a photographic sample book, open to a page containing a photograph. Give a detailed description of the photograph.'

In [ ]:
def get_response(url):
    image = Image.open(requests.get(url, stream=True).raw)
    image_tensor = process_images([image], image_processor, model.config)
    image_tensor = [_image.to(dtype=torch.float16, device=device) for _image in image_tensor]
    
    conv_template = "qwen_1_5"  # Make sure you use correct chat template for different models
    question = DEFAULT_IMAGE_TOKEN + f'\n{prompt}'
    conv = copy.deepcopy(conv_templates[conv_template])
    conv.append_message(conv.roles[0], question)
    conv.append_message(conv.roles[1], None)
    prompt_question = conv.get_prompt()
    
    input_ids = tokenizer_image_token(prompt_question, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(device)
    image_sizes = [image.size]
    
    cont = model.generate(
        input_ids,
        images=image_tensor,
        image_sizes=image_sizes,
        do_sample=False,
        temperature=0,
        max_new_tokens=1024,
    )
    text_outputs = tokenizer.batch_decode(cont, skip_special_tokens=True)

    return text_outputs

In [ ]:
responses_llava = {}

for url in urls:
    response = get_response(url)
    print(url,"\n",response,"\n\n")
    responses_llava[url] = {'lmms-lab/llava-onevision-qwen2-72b-ov':response}

In [ ]:
import pickle

In [ ]:
with open("responses_llava.pkl", "wb") as f:
    pickle.dump(responses_llava, f)